In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai
from openai import OpenAI
import os
import random
import json
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
client = OpenAI(api_key = )

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model = model,
        response_format={ "type": "json_object" },
        messages = messages)
    return response.choices[0].message.content

In [ ]:
data = pd.read_csv("/content/LIAR.csv", index_col = 0)
data.head(3)

,ID,label,statement,subjects,speaker,speaker_title,state,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,Justification
0,2635.json,FALSE,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...


In [ ]:
mul_label_mapping = {
    'pants-fire': 0,
    'FALSE': 1,
    'barely-true': 2,
    'half-true': 3,
    'mostly-true': 4,
    'TRUE': 5
}
bi_label_mapping = {
    'pants-fire': 0,
    'FALSE': 0,
    'barely-true': 0,
    'half-true': 0,
    'mostly-true': 1,
    'TRUE': 1
}
data['multiclass'] = data['label'].map(mul_label_mapping)
data['biclass'] = data['label'].map(bi_label_mapping)

Prompting: statement only

zero_shot_prompt = f"""Question: Given the following message, predict its veracity. If it's more likely to be a real message,return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}.Answer: $category_num""" \\

  zero_shot_CoT_prompt = f"""Question: Given the following message, predict its veracity. If it's more likely to be a real message,
  return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}
  Answer: Let's think step by step. """
  zero_shot_CoT_TD = f"""Question: Given the following messages, predict its varacity. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}.
  Answer: Let's think from perspective of textual description."""
  zero_shot_CoT_C = f"""Question: Given the following messages, predict its varacity. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}.
  Answer: Let's think from perspective of commonsense."""

In [ ]:
zs = []
zs_reason = []
zs_cot = []
zs_cot_reason = []
zs_cot_td = []
zs_cot_td_reason = []
zs_cot_c = []
zs_cot_c_reason = []
for i in tqdm(range(300)):
  statement = data['statement'][i]
  zero_shot_prompt = f""" Answer should be in JSON format whose keys are score and reason.
  Question: Given the following message, predict its veracity and tell me the reason. If it's more likely to be a real
  message,return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}.
  Answer: """
  zero_shot_CoT_prompt = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following message, predict its veracity and tell me the reason. If it's more likely to be a real message,
  return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}
  Answer: Let's think step by step. """
  zero_shot_CoT_TD = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following messages, predict its varacity and tell me the reason. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}.
  Answer: Let's think from perspective of textual description."""
  zero_shot_CoT_C = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following messages, predict its varacity and tell me the reason. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined: {statement}.
  Answer: Let's think from perspective of commonsense."""
  messages_zs = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_prompt}]
  messages_zs_CoT = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_prompt}]
  messages_zs_CoT_TD = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_TD}]
  messages_zs_CoT_C = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_C}]
  response_zs = json.loads(get_completion_from_messages(messages_zs, temperature=0))
  response_zs_CoT = json.loads(get_completion_from_messages(messages_zs_CoT, temperature=0))
  response_zs_CoT_TD = json.loads(get_completion_from_messages(messages_zs_CoT_TD, temperature=0))
  response_zs_CoT_C = json.loads(get_completion_from_messages(messages_zs_CoT_C, temperature=0))
  zs.append(response_zs['score'])
  zs_reason.append(response_zs['reason'])
  zs_cot.append(response_zs_CoT['score'])
  zs_cot_reason.append(response_zs_CoT['reason'])
  zs_cot_td.append(response_zs_CoT_TD['score'])
  zs_cot_td_reason.append(response_zs_CoT_TD['reason'])
  zs_cot_c.append(response_zs_CoT_C['score'])
  zs_cot_c_reason.append(response_zs_CoT_C['reason'])
  # print(zs_reason)
  # print(zs_cot_reason)
  # print(zs_cot_td_reason)
  # print(zs_cot_c_reason)

  0%|          | 0/300 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
LLM_response = pd.DataFrame({"zs":zs,"zs_cot": zs_cot,"zs_cot_td":zs_cot_td,"zs_cot_c":zs_cot_c,
                "zs_reason": zs_reason, "zs_cot_reason": zs_cot_reason, "zs_cot_td_reason": zs_cot_td_reason,
                "zs_cot_c_reason": zs_cot_c_reason})
result = pd.concat([data[['statement','label','biclass']].head(298), LLM_response], axis=1, ignore_index=False)

In [ ]:
result.to_csv("zs_result.csv")

In [ ]:
def metric(df, col):
  print(f"The confusion matrix for {col} is {confusion_matrix(df['biclass'], df[col])}")
  tn, fp, fn, tp = confusion_matrix(result['biclass'], result[col]).ravel()
  accuracy = round((tp+tn)/(tp+tn+fp+fn)*100,2)
  precision = round(tp/(tp+fp)*100,2)
  recall = round(tp/(tp+fn)*100,2)
  f1_score = round(2*precision*recall/(precision+recall),2)
  return accuracy, precision, recall,f1_score

In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs is [[92 92]
 [46 68]]
53.69 42.5 59.65 49.64


In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs_cot')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot is [[103  81]
 [ 61  53]]
52.35 39.55 46.49 42.74


In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs_cot_td')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot_td is [[ 65 119]
 [ 37  77]]
47.65 39.29 67.54 49.68


In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs_cot_c')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot_c is [[ 39 145]
 [ 13 101]]
46.98 41.06 88.6 56.11


Prompting statement + justificantion

In [ ]:
zs_js = []
zs_reason_js = []
zs_cot_js = []
zs_cot_reason_js = []
zs_cot_td_js = []
zs_cot_td_reason_js = []
zs_cot_c_js = []
zs_cot_c_reason_js = []
for i in tqdm(range(300)):
  statement = data['statement'][i]
  justification = data['Justification'][i]
  zero_shot_prompt = f""" Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checker, predict veracity of the news claim and tell me the reason. If it's more likely to be a real
  message,return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  Answer: """
  zero_shot_CoT_prompt = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checker, predict veracity of the news claim and tell me the reason. If it's more likely to be a real message,
  return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  Answer: Let's think step by step. """
  zero_shot_CoT_TD = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checker, predict veracity of the news claim and tell me the reason. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  Answer: Let's think from perspective of textual description."""
  zero_shot_CoT_C = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checker, predict veracity of the news claim and tell me the reason. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  Answer: Let's think from perspective of commonsense."""
  messages_zs = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_prompt}]
  messages_zs_CoT = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_prompt}]
  messages_zs_CoT_TD = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_TD}]
  messages_zs_CoT_C = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_C}]
  response_zs = json.loads(get_completion_from_messages(messages_zs, temperature=0))
  response_zs_CoT = json.loads(get_completion_from_messages(messages_zs_CoT, temperature=0))
  response_zs_CoT_TD = json.loads(get_completion_from_messages(messages_zs_CoT_TD, temperature=0))
  response_zs_CoT_C = json.loads(get_completion_from_messages(messages_zs_CoT_C, temperature=0))
  zs_js.append(response_zs['score'])
  zs_reason_js.append(response_zs['reason'])
  zs_cot_js.append(response_zs_CoT['score'])
  zs_cot_reason_js.append(response_zs_CoT['reason'])
  zs_cot_td_js.append(response_zs_CoT_TD['score'])
  zs_cot_td_reason_js.append(response_zs_CoT_TD['reason'])
  zs_cot_c_js.append(response_zs_CoT_C['score'])
  zs_cot_c_reason_js.append(response_zs_CoT_C['reason'])
  # if i % 10 == 0:
  #   print(len(zs_js))
  #   print(len(zs_cot_c_reason_js))

100%|██████████| 300/300 [19:21<00:00,  3.87s/it]


In [ ]:
LLM_response_js = pd.DataFrame({"zs":zs_js,"zs_cot": zs_cot_js,"zs_cot_td":zs_cot_td_js,"zs_cot_c":zs_cot_c_js,
                "zs_reason": zs_reason_js, "zs_cot_reason": zs_cot_reason_js, "zs_cot_td_reason": zs_cot_td_reason_js,
                "zs_cot_c_reason": zs_cot_c_reason_js})
result_js = pd.concat([data[['statement','label','biclass']].head(300), LLM_response_js], axis=1, ignore_index=False)

In [ ]:
accuracy, precision, recall,f1_score= metric(result_js,'zs')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs is [[114  70]
 [ 42  74]]
53.69 42.5 59.65 49.64


In [ ]:
accuracy, precision, recall,f1_score= metric(result_js,'zs_cot')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot is [[134  50]
 [ 80  36]]
52.35 39.55 46.49 42.74


In [ ]:
accuracy, precision, recall,f1_score= metric(result_js,'zs_cot_td')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot_td is [[87 97]
 [46 70]]
47.65 39.29 67.54 49.68


In [ ]:
accuracy, precision, recall,f1_score= metric(result_js,'zs_cot_c')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot_c is [[ 58 126]
 [ 29  87]]
46.98 41.06 88.6 56.11


In [ ]:
result_js.to_csv('S+J.csv')

In [ ]:
data.columns

Index(['ID', 'label', 'statement', 'subjects', 'speaker', 'speaker_title',
       'state', 'party_affiliation', 'barely_true_counts', 'false_counts',
       'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts',
       'context', 'Justification', 'multiclass', 'biclass'],
      dtype='object')

In [ ]:
data[['statement', 'subjects', 'speaker', 'speaker_title',
       'state', 'party_affiliation','context']].head(3)

,statement,subjects,speaker,speaker_title,state,party_affiliation,context
0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,a mailer
1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.
2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,Denver


In [ ]:
data[['barely_true_counts', 'false_counts',
       'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts']].mean(axis=0)

barely_true_counts      11.583939
false_counts            13.359059
half_true_counts        17.185785
mostly_true_counts      16.497850
pants_on_fire_counts     6.251388
dtype: float64

Prompting: statement + justification + metadata

In [ ]:
zs_jsm = []
zs_reason_jsm = []
zs_cot_jsm = []
zs_cot_reason_jsm = []
zs_cot_td_jsm = []
zs_cot_td_reason_jsm = []
zs_cot_c_jsm = []
zs_cot_c_reason_jsm = []
for i in tqdm(range(300)):
  statement = data['statement'][i]
  justification = data['Justification'][i]
  subject = data['subjects'][i]
  context = data['context'][i]
  speaker = data['speaker'][i]
  job_title = data['speaker_title'][i]
  state = data['state'][i]
  party = data['party_affiliation'][i]
  pants_on_fire_cnt = data['pants_on_fire_counts'][i]
  false_cnt = data['false_counts'][i]
  barely_true_cnt = data['barely_true_counts'][i]
  half_true_cnt = data['half_true_counts'][i]
  most_true_cnt = data['mostly_true_counts'][i]
  fake_cnt = pants_on_fire_cnt + false_cnt + barely_true_cnt + half_true_cnt

  zero_shot_prompt = f""" Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim, justification of other fact-checkers and other relevant information about
  the news claim and its speaker, you need to predict veracity of the news claim and tell me the reason. If it's more likely to be a real
  message,return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  SUBJECT_OF_CLAIM: {subject}
  SOURCE_OF_CLAIM: {context}
  INFORMATION_OF_CLAIM_SPEAKER: {speaker} is a {job_title} from {state}. In the past, he or she has {fake_cnt} news marked as fake news.
  Sort by the degree of falsehood from highest to lowest, there are {pants_on_fire_cnt} news labeled as 'pants on fire' (market avg is 6);
  {false_cnt} news labeled as 'false' (market avg is 13),{barely_true_cnt} news labeled as 'barely true' (market avg is 11.5),
  {half_true_cnt} news labeled as 'half true' (market avg is 17), {most_true_cnt} labeled as 'mostly true' (market avg is 16);
  Answer: """
  zero_shot_CoT_prompt = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checkers and other relevant information about
  the news claim and its speaker, you need to predict veracity of the news claim and tell me the reason. If it's more likely to be a real message,
  return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  SUBJECT_OF_CLAIM: {subject}
  SOURCE_OF_CLAIM: {context}
  INFORMATION_OF_CLAIM_SPEAKER: {speaker} is a {job_title} from {state}. In the past, he or she has {fake_cnt} news marked as fake news.
  Sort by the degree of falsehood from highest to lowest, there are {pants_on_fire_cnt} news labeled as 'pants on fire' (market avg is 6);
  {false_cnt} news labeled as 'false' (market avg is 13),{barely_true_cnt} news labeled as 'barely true' (market avg is 11.5),
  {half_true_cnt} news labeled as 'half true' (market avg is 17), {most_true_cnt} labeled as 'mostly true' (market avg is 16).
  Answer: Let's think step by step. """
  zero_shot_CoT_TD = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checker and other fact-checkers and other relevant information about
  the news claim and its speaker, you need to predict veracity of the news claim and tell me the reason. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  SUBJECT_OF_CLAIM: {subject}
  SOURCE_OF_CLAIM: {context}
  INFORMATION_OF_CLAIM_SPEAKER: {speaker} is a {job_title} from {state}. In the past, he or she has {fake_cnt} news marked as fake news.
  Sort by the degree of falsehood from highest to lowest, there are {pants_on_fire_cnt} news labeled as 'pants on fire' (market avg is 6);
  {false_cnt} news labeled as 'false' (market avg is 13),{barely_true_cnt} news labeled as 'barely true' (market avg is 11.5),
  {half_true_cnt} news labeled as 'half true' (market avg is 17), {most_true_cnt} labeled as 'mostly true' (market avg is 16).
  Answer: Let's think from perspective of textual description."""
  zero_shot_CoT_C = f"""Answer should be in JSON format whose keys are score and reason.
  Question: Given the following news claim and justification of other fact-checker and other fact-checkers and other relevant information about
  the news claim and its speaker, you need to predict veracity of the news claim and tell me the reason. If it is more likely to be a real
  message, return 1; otherwise, return 0. Please refrain from providing ambiguous assessments such as undetermined.
  NEWS_CLAIM: {statement};
  JUSTIFICATION: {justification}
  SUBJECT_OF_CLAIM: {subject}
  SOURCE_OF_CLAIM: {context}
  INFORMATION_OF_CLAIM_SPEAKER: {speaker} is a {job_title} from {state}. In the past, he or she has {fake_cnt} news marked as fake news.
  Sort by the degree of falsehood from highest to lowest, there are {pants_on_fire_cnt} news labeled as 'pants on fire' (market avg is 6);
  {false_cnt} news labeled as 'false' (market avg is 13),{barely_true_cnt} news labeled as 'barely true' (market avg is 11.5),
  {half_true_cnt} news labeled as 'half true' (market avg is 17), {most_true_cnt} labeled as 'mostly true' (market avg is 16).
  Answer: Let's think from perspective of commonsense."""
  messages_zs = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_prompt}]
  messages_zs_CoT = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_prompt}]
  messages_zs_CoT_TD = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_TD}]
  messages_zs_CoT_C = [
      {"role": "system", "content": "As a talented fact-checker in journalism who will evaludate the veracity and truthfulness \
      of the given news claim and answer the veraity score following users' instruction. Your judgments should be \
      supported by evidence and logical reasoning. Please consider any potential biases or uncertainties in the \
      provided information and address them in your evaluation.",
      "role": "user", "content": zero_shot_CoT_C}]
  response_zs = json.loads(get_completion_from_messages(messages_zs, temperature=0))
  response_zs_CoT = json.loads(get_completion_from_messages(messages_zs_CoT, temperature=0))
  response_zs_CoT_TD = json.loads(get_completion_from_messages(messages_zs_CoT_TD, temperature=0))
  response_zs_CoT_C = json.loads(get_completion_from_messages(messages_zs_CoT_C, temperature=0))
  zs_jsm.append(response_zs['score'])
  zs_reason_jsm.append(response_zs['reason'])
  zs_cot_jsm.append(response_zs_CoT['score'])
  zs_cot_reason_jsm.append(response_zs_CoT['reason'])
  zs_cot_td_jsm.append(response_zs_CoT_TD['score'])
  zs_cot_td_reason_jsm.append(response_zs_CoT_TD['reason'])
  zs_cot_c_jsm.append(response_zs_CoT_C['score'])
  zs_cot_c_reason_jsm.append(response_zs_CoT_C['reason'])



100%|██████████| 300/300 [32:15<00:00,  6.45s/it]


In [ ]:
LLM_response = pd.DataFrame({"zs":zs_jsm,"zs_cot": zs_cot_jsm,"zs_cot_td":zs_cot_td_jsm,"zs_cot_c":zs_cot_c_jsm,
                "zs_reason": zs_reason_jsm, "zs_cot_reason": zs_cot_reason_jsm, "zs_cot_td_reason": zs_cot_td_reason_jsm,
                "zs_cot_c_reason": zs_cot_c_reason_jsm})
result = pd.concat([data[['statement','label','biclass']].head(300), LLM_response], axis=1, ignore_index=False)

In [ ]:
result.to_csv("result_sjm.csv")

In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs is [[126  58]
 [ 71  45]]
57.0 43.69 38.79 41.09


In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs_cot')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot is [[108  76]
 [ 53  63]]
57.0 45.32 54.31 49.41


In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs_cot_td')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot_td is [[109  75]
 [ 58  58]]
55.67 43.61 50.0 46.59


In [ ]:
accuracy, precision, recall,f1_score= metric(result,'zs_cot_c')
print(accuracy, precision, recall,f1_score)

The confusion matrix for zs_cot_c is [[ 80 104]
 [ 31  85]]
55.0 44.97 73.28 55.74
